In [5]:
import pandas as pd
from census_api import census_df
from scrape_realtor import realtor_df

In [6]:
print(realtor_df.head())

  month_date_yyyymm county_fips          county_name  median_listing_price  \
0            202208       38015         burleigh, nd                382450   
1            202208       28147         walthall, ms                169250   
2            202208       17153          pulaski, il                 84700   
3            202208       51730  petersburg city, va                194995   
4            202208       26061         houghton, mi                254450   

   active_listing_count  median_days_on_market  new_listing_count  \
0                   225                     39                136   
1                    13                     51                  4   
2                     9                     67                  4   
3                   117                     36                 84   
4                    50                     37                 36   

   price_increased_count  price_reduced_count  pending_listing_count  \
0                      0                   8

In [8]:
print(census_df)


                County       State  Total:  Total:Population of one race:  \
0       Autauga County     Alabama   58805                          55648   
1       Baldwin County     Alabama  231767                         216743   
2       Barbour County     Alabama   25223                          24523   
3          Bibb County     Alabama   22293                          21534   
4        Blount County     Alabama   59134                          55478   
...                ...         ...     ...                            ...   
3216   Renville County   Minnesota   14723                          13925   
3217     Roseau County   Minnesota   15331                          14626   
3218  Sherburne County   Minnesota   97183                          92120   
3219     Steele County   Minnesota   37406                          35421   
3220       Todd County   Minnesota   25262                          24109   

      Total:Population of one race:White alone  \
0                        